# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [2]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

r = requests.get(url)
dados_json = r.json()
dados_df = pd.DataFrame(dados_json['items'])
dados_df.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,26368,Município,Rio de Janeiro,3304557,RJ,PVL02.002098/2017-76,Deferido,17944.102249/2017-58,2017-11-29T19:03:49Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.000000e+08,1,0,04/12/2017
1,9452,Município,Rio de Janeiro,3304557,RJ,None,Arquivado,17944.000984/2009-18,2009-08-13T03:00:00Z,Operação contratual externa (com garantia da U...,Multissetorial,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,1.000000e-02,0,0,09/02/2010
2,9664,Município,Rio de Janeiro,3304557,RJ,None,Deferido,17944.001028/2010-97,2011-04-14T03:00:00Z,Operação contratual interna,RELUZ,Empresa Não Estatal,Light - Serviços de Eletricidade S/A,Real,1.172168e+07,1,0,12/05/2011
3,10230,Município,Resende,3304201,RJ,None,Arquivado,17944.001150/2008-49,2008-07-18T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,5.734000e+05,0,0,02/10/2008
4,10259,Município,Armação dos Búzios,3300233,RJ,None,Regularizado,17944.001156/2009-05,2009-11-11T02:00:00Z,Operação contratual interna,Regularização de Dívida - Água e Esgoto,Empresa Não Estatal,Prolagos S/A - Concessionária de Serviços Públ...,Real,7.684211e+04,0,0,30/11/2009


In [3]:
# 2) Seu código aqui
dados_df['status'].value_counts().head(3)

Deferido        86
Arquivado       53
Regularizado    25
Name: status, dtype: int64

In [4]:
# 3) Seu código aqui
dados_df['ano_status'] = dados_df['data_status'].apply(lambda i: str(i.split('/')[-1]))
dados_df['ano_status'].head()

0    2017
1    2010
2    2011
3    2008
4    2009
Name: ano_status, dtype: object

In [5]:
# 3) Alternativo
dados_df['data_status'] =  pd.to_datetime(dados_df['data_status'])
dados_df['ano_status_2'] = dados_df['data_status'].apply(lambda i: i.year)
dados_df['ano_status_2'].head()

0    2017
1    2010
2    2011
3    2008
4    2009
Name: ano_status_2, dtype: int64

In [6]:
# 4)
dados_df['ano_status'].value_counts()

2008    37
2007    29
2014    20
2010    19
2019    17
2020    15
2013    15
2012    13
2011    10
2015     9
2009     9
2017     5
2021     5
2016     4
2018     4
2002     4
2006     4
2004     1
2003     1
Name: ano_status, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [7]:
#1) Seu código aqui
def pegar_dados_api(uf: str, tipo: str):
    url_i = f'http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo}'  

    r_i = requests.get(url_i)
    dados_json_i = r_i.json()
    dados_i = pd.DataFrame(dados_json_i['items'])
    return dados_i
    

In [8]:
# 2) Seu código aqui
df = pegar_dados_api('MG', 'Estado')
len(df[df['status'] == 'Arquivado por decurso de prazo'])

1

In [9]:
# 3) Seu código aqui
df = pegar_dados_api('BA', 'Município')
df[df['status'] == 'Deferido']['interessado'].value_counts(ascending=False).head(1)

Luís Eduardo Magalhães    14
Name: interessado, dtype: int64

In [10]:
# 4) Seu código aqui
df = pegar_dados_api('BA', 'Estado')
df.to_csv('dados_solicitacoes_BA.csv', index=False)